# **Introduction:** 

This file serves to host the downstream integration of our project for TME 6015: AI/ML Workflow Design. The integration that was used pertains to a simulated task allocation scenario within a warehouse environment. Tasks within the warehouse can have three different load requirements: *Small*, *Medium*, or *Large*, and robots can have a similar load capacity denoted by *Small*, *Medium*, or *Large*.

Robots are also defined by their distance to the task, their load history, and their travel history. The goal is therefore to:

* Allocate a robot with a matching capability to the task requirement;
* Prioritize robots with a low load history;
* Prioritize robots that are closer to the task;
* Prioritize robots that have not travelled far at the time of allocation, and;
* To accomplish the aforementioned goals using our ANN architecture. 

**Date Created: 25/11/2024**

**Date Modified: 25/11/2024**

# **Import Packages:**

Need to import the following packages for this simulation:

In [195]:
# import packages:

import numpy as np
import matplotlib.pyplot as plt
import math as m
import heapq
import cv2
import os
import time
from PythonFISFunction import *
from IPython.display import clear_output

# **Define Classes & Functions:**

Need to define the following classes and functions for this simulation:

In [196]:
# define robot class:

class Robot:
    # constructor:
    def __init__(self, id, capability, position, colour):   
        self.id = id                      # identifier tag for each robot
        self.capability = capability      # carrying capacity of the robot
        self.position = position          # position in space of robot
        self.colour = colour              # colour identifier
        self.initial_position = position  # initial position used when switching between FIS and ANN

        self.load_history = 0.0           # load history of the robot
        self.distance_to_task = 0.0       # planned travel distance of robot
        self.travel_history = 0.0         # travel history of the robot
        self.cap_match = 0.0              # binary capability matching of each robot
        
        self.suitability = 0.0            # suitability of robot

    # query:
    def display_robot_info(self):
        return (f"Robot ID: {self.id}\n"
                f"Position: {self.position}\n"
                f"Capability Type: {self.capability}\n"
                f"Load History: {self.load_history}\n"
                f"Travelled Distance: {self.travel_history}\n"
                f"Suitability: {self.suitability}")

class Task:
    # constructor:
    def __init__(self, position, requirement, colour):
        self.position = position
        self.requirement = requirement
        self.colour = colour

def dijkstra(image, start, goal):

    # first need to get the dimensionality of the image:
    rows, cols = image.shape

    # swap the (x, y) input to (y, x) format for internal use - cv2 has y,x notation
    start = (start[1], start[0])  # swap (x, y) -> (y, x)
    goal = (goal[1], goal[0])     # swap (x, y) -> (y, x)

    # need to initialize both the distance map and the previous node map:
    dist = np.full((rows,cols), np.inf)  # set other distances to a very big number
    dist[start] = 0                      # set the initial starting distance to 0

    # need to track the parent of each node such that the resulting path can be reconstructed:
    parent = {start: None}

    # start the priority queue to store the distance values in x and y:
    pq = [(0, start)]

    # encode the directions that the robot can move, assuming 8 options of movement at each given
    # node, 45 degree offsets (holonomic movement):
    directions = [
        (-1,0), (1,0), (0,-1), (0,1),    # left, right, down, up
        (-1,-1), (-1,1), (1,-1), (1,1)   # diagonals
    ]

    # define a set for the visited node:
    visited = set()

    while pq:
        current_dist, (x, y) = heapq.heappop(pq)
        
        # if the node has already been visited, skip it
        if (x, y) in visited:
            continue
        visited.add((x, y))
        
        # if we reached the goal, reconstruct the path:
        if (x, y) == goal:
            path = []
            while (x,y) != start:
                path.append((y,x))
                x,y = parent[(x,y)]
            path.append((start[1],start[0]))
            return path[::-1], dist[goal] # reverse path 
        
    # explore neighbors
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and (nx, ny) not in visited:
                # ignore black borders
                if image[nx, ny] >= 254:  # threshold for white space
                    # calculate the movement cost
                    movement_cost = m.sqrt(2) if dx != 0 and dy != 0 else 1
                    new_dist = current_dist + movement_cost
                    
                    # if a shorter path is found, update the distance and push to pq
                    if new_dist < dist[nx, ny]:
                        dist[nx, ny] = new_dist
                        parent[(nx,ny)] = (x,y)
                        heapq.heappush(pq, (new_dist, (nx, ny)))
    
    return None, None # if goal unreachable

def add_buffer(image, buffer_size):

    # create a binary mask where black (0) and gray (205) areas are marked
    mask = np.where((image == 0) | (image == 205), 1, 0).astype(np.uint8)
    
    # create a kernel for dilation (buffer expansion)
    kernel = np.ones((buffer_size, buffer_size), np.uint8)
    
    # dilate the mask (expand obstacles)
    dilated_mask = cv2.dilate(mask, kernel, iterations=1)
    
    # create a new image where dilated areas are treated as non-navigable (set to black)
    buffered_image = image.copy()
    buffered_image[dilated_mask == 1] = 0  # set dilated areas to black (0)

    # white space detection:
    _, thresh = cv2.threshold(buffered_image, 254, 255, cv2.THRESH_BINARY)

    # Find coordinates of white pixels
    white_pixels = cv2.findNonZero(thresh)
    
    return buffered_image, white_pixels

def draw_circles_on_image(image, task):
    for robot in robots.values():
        position = robot.position

        cv2.circle(image, position, 3 , robot.colour, -1)
        cv2.circle(image, task.position, 3, task.colour, -1)
    
    return image

# **Initializing the Map:**

Need to first load the map and determine the white space where robots can be spawned:

In [197]:
# get the map string:
files_in_dir = os.listdir(os.getcwd())
file_path = os.path.join(os.getcwd(), files_in_dir[files_in_dir.index('warehouse_map.png')])

# load the image:
image = cv2.imread(file_path,0)
image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

# apply dilation to get navigable space assuming 30 cm (6 pixel x 5cm/pixel) tolerance:
buffered_image, spawn_locations = add_buffer(image, 6)

Set the parameters for the simulation. This defines the total number of robots, and the number of each load capacity to include:

In [198]:
# reshape spawn locations:
spawn_locations = spawn_locations.reshape(-1, 2)

# define simulation parameters:
nr = 6      # number of robots overall
x = 2       # nunber of "small" robots
y = 2       # number of "medium" robots
z = 2       # number of "large" robots

nt = 1     # number of tasks to be spawned

robots = {}               # empty dictionary to hold robots
tasks = {}                # empty dictionary to hold tasks

rulebase = fis_create()   # initialize rulebase for the FIS
# load model here


Spawn robots based on parameters:

In [199]:
# spawn robots:
for num in range(1, nr+1):
    robot_name = f"Robot {num}"

    # if we haven't exceeded the number of "small" robots:
    if num <= x:
        index = np.random.choice(spawn_locations.shape[0])
        robots[robot_name] = Robot(
            id = num,
            capability = 'Small',
            position = spawn_locations[index],
            colour = (0,255,0)
        )
        spawn_locations = np.delete(spawn_locations, index, axis = 0)
    
    # if we haven't exceeded the number of "medium" robots:
    elif num  <= y+x:
        index = np.random.choice(spawn_locations.shape[0])
        robots[robot_name] = Robot(
            id = num,
            capability = 'Medium',
            position = spawn_locations[index],
            colour = (0,0,255)
        )
        spawn_locations = np.delete(spawn_locations, index, axis = 0)

    # if we haven't exceeded the number of "large" robots:
    else:
        index = np.random.choice(spawn_locations.shape[0])
        robots[robot_name] = Robot(
            id = num,
            capability = 'Large',
            position = spawn_locations[index],
            colour = (255,165,0)
        )
        spawn_locations = np.delete(spawn_locations, index, axis = 0)

Generate as many tasks as the user specifies:

In [200]:
# spawn tasks:
for i in range(1, nt+1):

    task_name = f"Task {i}"

    # assign task object a random position within space:
    index = np.random.choice(spawn_locations.shape[0])
    task_position = spawn_locations[index]
    spawn_locations = np.delete(spawn_locations, index, axis = 0)

    # assign task object a random requirement:
    task_requirement = np.random.choice(['Small', 'Medium', 'Large'])

    # make task red:
    task_colour = (255,0,0)

    # create task object
    task = Task(task_position, task_requirement, task_colour)

    # add to dictionary
    tasks[task_name] = task


# **Simulation - FIS:**

Can now simulate task allocation using the FIS. This is done by first iterating through each task and having each robot calculate their suitability. The robot with the highest suitability goes to the task site, and has their parameters updated accordingly.

In [ ]:
# allocation simulation - FIS:

fig = plt.figure(figsize = (12,10))

for task in tasks.values():

    # initial plot to view positions:
    combined_image = draw_circles_on_image(image_rgb.copy(), task)

    clear_output(wait=True)
    fig = plt.figure(figsize = (12,10))
    plt.imshow(combined_image)
    plt.pause(1)

    for id, robot in robots.items():

        # get robot position:
        start = robot.position

        # determine length of planned path:
        shortest_path, dist = dijkstra(buffered_image, start, task.position)

        # add the shortest path if it exists:
        if shortest_path is not None:
            for x,y in shortest_path:
                combined_image[y,x] = robot.colour

        # update the robots planned travel distance:
        robot.distance_to_task = round((dist*0.05), 3)

        # need to get capability match:
        robot.cap_match = 1 if task.requirement == robot.capability else 0

        # use FIS to get suitability:
        robot.suitability = round(fis_solve(rulebase, robot.load_history, robot.distance_to_task, robot.travel_history, robot.cap_match), 2)
        # print(robot.suitability)

        # get ID of highest suitability robot:

        best_robot_id = max(robots, key=lambda robot_id: robots[robot_id].suitability)
    
    # draw with planned path:
    clear_output(wait=True)
    fig = plt.figure(figsize = (12,10))
    plt.imshow(combined_image)
    plt.pause(1)

    # best robot goes to task site, update their parameters accordingly:
    best_robot = robots[best_robot_id]
    best_robot.load_history += 1
    best_robot.position = task.position
    best_robot.travel_history = robot.distance_to_task

    # draw after allocation:
    task.position = (0,0)
    combined_image = draw_circles_on_image(image_rgb.copy(),task)

    clear_output(wait=True)
    fig = plt.figure(figsize = (12,10))
    plt.imshow(combined_image)
    plt.pause(1)
    

# **Simulation - ANN:**

Need to first reset the robot positions to their earlier states and redo the simulation with the ANN.